In [2]:
import numpy as np 
from qiskit import QuantumRegister, ClassicalRegister,QuantumCircuit, transpile, Aer
from qiskit.providers.aer import QasmSimulator
%matplotlib inline

In [3]:
def num_to_binary(n,N):
    # n is the number to be represented in binary
    # N is the number of digits of the rapresentation
    #es: num_to_binary(5,4) = 0101
    Nbits=2**N
    if n>=Nbits: return 0
    return bin(n+2*Nbits)[4:]


def HammingCircuit(N,ClassicalRegisters=None,ancillas=1):
    """
    -Returns a circuit with just the qbits labeled as the parity and signal
    -it gives you a circuit with 2^N qbits of (message + redundancy)
    -it can have classical registers if you want set ClassicalRegisters=True, if given a int "n" it will have 
    n classical registers
    by default it will have N ancillas, if specified it will have a numer N*ancillas"""
    registers=[]
    for i in range(2**N):
        prefix='s' #s stands for signal
        if i==0 or np.log2(i)==int(np.log2(i)): prefix='p' #c stands for parity
        registers.append(QuantumRegister(1,prefix+num_to_binary(i, N)))
    if ClassicalRegisters!=None: 
        if ClassicalRegisters==True: registers.append(ClassicalRegister(2**N+N*ancillas))
        else: registers.append(ClassicalRegister(ClassicalRegisters))
    circuit=QuantumCircuit(*registers,QuantumRegister(N*ancillas,'anc')) #circuit already with ancillas
    #circit=QuantumCircuit(*registers)
    circuit.N=N
    return circuit


def xor(N):
    #This is the gate that calculates the xor of all the position with ones, this gives the position of the faulty qbit
    circ=HammingCircuit(N)
    nqubits=2**N
    for i in range(1,nqubits):
        for j in range(0,N):
            if i & 2**j == 2**j: circ.cx(i,nqubits+j)
    return circ.to_gate(label='Initialize')

def correct(N):
    #This is the gate that corrects the faulty qbit
    circ=HammingCircuit(N)
    nqubits=2**N
    count=np.zeros(N)
    for i in range(1,nqubits):
        count=count+1
        for j in range(N):
            count[j]=count[j]%(2**j)
            if count[j]==0 and i!=1: circ.x(nqubits+j)
            if i==1 and j!=0: circ.x(nqubits+j)
        circ.mct([*range(nqubits,nqubits+N)],i)
    return circ.to_gate(label='Correction')

def HammingGate0(N):
    circ=HammingCircuit(N)
    circ.append(xor(N),[*range(2**N+N)])
    circ.append(correct(N),[*range(2**N+N)])
    return circ.to_gate(label='Hamming0')

In [4]:
simulator = QasmSimulator()

Initializing Ancillas

In [5]:
N=3
circuit=HammingCircuit(N,True)
asd=HammingGate0(N)
circuit.append(asd,[*range(2**N+N)])
circuit.measure([*range(2**N+N)],[*range(2**N+N)])
circuit.draw()

┌────────────┐┌─┐                              
p000_0: ┤0           ├┤M├──────────────────────────────
        │            │└╥┘┌─┐                           
p001_0: ┤1           ├─╫─┤M├───────────────────────────
        │            │ ║ └╥┘┌─┐                        
p010_0: ┤2           ├─╫──╫─┤M├────────────────────────
        │            │ ║  ║ └╥┘┌─┐                     
s011_0: ┤3           ├─╫──╫──╫─┤M├─────────────────────
        │            │ ║  ║  ║ └╥┘┌─┐                  
p100_0: ┤4           ├─╫──╫──╫──╫─┤M├──────────────────
        │            │ ║  ║  ║  ║ └╥┘┌─┐               
s101_0: ┤5  Hamming0 ├─╫──╫──╫──╫──╫─┤M├───────────────
        │            │ ║  ║  ║  ║  ║ └╥┘┌─┐            
s110_0: ┤6           ├─╫──╫──╫──╫──╫──╫─┤M├────────────
        │            │ ║  ║  ║  ║  ║  ║ └╥┘┌─┐         
s111_0: ┤7           ├─╫──╫──╫──╫──╫──╫──╫─┤M├─────────
        │            │ ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
 anc_0: ┤8           ├─╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────
        │            │ ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
 anc_1: ┤9           ├─╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
        │            │ ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
 anc_2: ┤10          ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
        └────────────┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
 c0: 11/═══════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                       0  1  2  3  4  5  6  7  8  9  10

In [57]:
def HammingRedundant(n):
    bits=int(np.log2(n))+1
    parity=int(np.log2(bits))+2
    total=2**(parity-1)
    N,j=0,0
    for i in range(total):
        if i!=0 and int(np.log2(i))!=np.log2(i):
            N=N+(n&(2**j)==2**j)*(2**i)
            j=j+1
    print(num_to_binary(N,total))
    for i in range(parity):
        i=2**i
        par=0
        for j in range(total):
            temp=N
            N=N^((2**i)*((j&i==i)&((N&2**j)!=0)))
    return N
N=14
Ham=HammingRedundant(N)
print(num_to_binary(N,4),num_to_binary(Ham,8))

11100000
1110 11101000


In [11]:
compiled_circuit = transpile(circuit, simulator)
job = simulator.run(compiled_circuit, shots=1000)
result = job.result()
counts = result.get_counts(circuit)
print('asfafg',counts)

asfafg {'00000000000': 1000}


In [33]:
195&2**7

128